In [ ]:

    
#!/usr/bin/env python
# coding: utf-8


import os
import sys
import re
import torch
from glob import glob
from openbabel import pybel
import numpy as np
import random
import pickle
import warnings
import requests
import os
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
from sklearn.model_selection import train_test_split
from dask.dataframe import from_pandas
from dask.multiprocessing import get
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False, nb_workers=250,use_memory_fs=False)


PYTORCH_ENABLE_MPS_FALLBACK=1
PYTORCH_ENABLE_SparseCPU_FALLBACK=1
warnings.filterwarnings("ignore")

from dask_jobqueue import SLURMCluster

types_of_aa=[]

train_grids=None
test_grids=None
rotations=9
full_batch=10
features_shape=94
# First training grids: 
train_label=[]
# Second testing grids: 
test_label=[]
    
# cluster = SLURMCluster(cores=1024,
#                        processes=12,
#                        memory="250GB",
#                       # account="hmslati",
#                       # walltime="01:00:00",
#                        queue="gpu-bigmem")

# PLEASE READ -> 45次实验分别进行10倍交叉验证，取平均

#Converts the protein-ligand complexes into 4D tensor. 
class Feature_extractor():
    def __init__(self):
        self.atom_codes = {}
        #'others' includs metal atoms and B atom. There are no B atoms on training and test sets. 
        # 89 to 102 will be reserved to PLIF features as follows:
        # 89: hydrophobic
        # 90: hbond
        # 91: waterbridge
        # 92: saltbridge
        # 93: pistacking
        # 94: pication
        # 95: halogen
        # 96: metal
        # 97: Distances 
        
        # rarely occuring elements: ['4', '13', '22-23', '25', '28', '32', '40', '45', '47-52', '55', '75-76', '78', '80', '82'] 
        others = ([4,13,25,28,32,40,45,55,78,80,82]+list(range(22,24))+list(range(47,53))+list(range(75,77)))
        plif_specs=list(range(89,98))
        #C and N atoms can be hybridized in three ways and S atom can be hybridized in two ways here. 
        #Hydrogen atom is also considered for feature extraction. I think phosphor atom has 3 or 5 as hyb states but 
        # in biological system its usually the same recurrent phosphate even in most small molecules so safe to assume one
        # hybridization state for this purpose. 
        atom_types = [1,(6,1),(6,2),(6,3),(7,1),(7,2),(7,3),8,15,(16,2),(16,3),
                      34,9,17,35,53,11,12,13,14,5,19,20,25,29,28,30,33,3,27,24,26,31,42,79,44,74,others]+plif_specs
      
        for i, j in enumerate(atom_types):
            if type(j) is list:
                for k in j:
                    self.atom_codes[k] = i
                
            else:
                self.atom_codes[j] = i              
        
        self.sum_atom_types = len(atom_types)
        
    #Onehot encoding of each atom. The atoms in protein or ligand are treated separately.
    def encode(self, atomic_num, orig_coords, plifs, molprotein):
        encoding = np.zeros(self.sum_atom_types*2)
        if molprotein == 1:
            encoding[self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.atom_codes[89]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.atom_codes[90]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.atom_codes[91]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.atom_codes[92]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.atom_codes[93]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.atom_codes[94]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.atom_codes[95]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.atom_codes[96]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.atom_codes[97]] = plifs[coord][1]
                    

                
        else:
            encoding[self.sum_atom_types+self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.sum_atom_types+self.atom_codes[89]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[90]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[91]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[92]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[93]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[94]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[95]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[96]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.sum_atom_types+self.atom_codes[97]] = plifs[coord][1]

        
        return encoding
    
    #Get atom coords and atom features from the complexes.   
    def get_features(self, molecule, plifs, molprotein):
        coords = []
        features = []
            
        for atom in molecule:
            coords.append(atom.coords)
            if atom.atomicnum in [6,7,16]:
                atomicnum = (atom.atomicnum,atom.hyb)
                features.append(self.encode(atomicnum,atom.coords,plifs,molprotein))
            else:
                features.append(self.encode(atom.atomicnum,atom.coords,plifs,molprotein))
        
        coords = np.array(coords, dtype=np.float32)
        features = np.array(features, dtype=np.float32)

        return coords, features
     
    #Define the rotation matrixs of 3D stuctures.
    def rotation_matrix(self, t, roller):
        if roller==0:
            return np.array([[1,0,0],[0,np.cos(t),np.sin(t)],[0,-np.sin(t),np.cos(t)]])
        elif roller==1:
            return np.array([[np.cos(t),0,-np.sin(t)],[0,1,0],[np.sin(t),0,np.cos(t)]])
        elif roller==2:
            return np.array([[np.cos(t),np.sin(t),0],[-np.sin(t),np.cos(t),0],[0,0,1]])

    #Generate 3d grid or 4d tensor. Each grid represents a voxel. Each voxel represents the atom in it by onehot encoding of atomic type.
    #Each complex in train set is rotated 9 times for data amplification.
    #The complexes in core set are not rotated. 
    #The default resolution is 20*20*20.
    def grid(self,grid, coords, features, frag_idx, resolution=1.0, max_dist=10.0,  rotation_bool=True, max_frag=10, rotations=9):
        assert coords.shape[1] == 3
        assert coords.shape[0] == features.shape[0]  

        slider=frag_idx*20

        x=y=z=np.array(range(-10,10),dtype=np.float32)+0.5
        u=0
        for i in range(len(coords)):
            coord=coords[i]
            # add/subtract 10 from the center
            tmpx=abs(coord[0]-x)
            tmpy=abs(coord[1]-y)
            tmpz=abs(coord[2]-z)

            if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                u+=1
                # get the position of the closest point to coordinate which is found inside the grid
                # append the features unto that slice

                grid[0,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += torch.tensor(features[i])
                
        if rotation_bool:
            for rotation_idx in range(rotations):
                theta = random.uniform(np.pi/18,np.pi/2)
                roller = random.randrange(3)
                coords = np.dot(coords, self.rotation_matrix(theta,roller))
                for i in range(len(coords)):
                    coord=coords[i]
                    tmpx=abs(coord[0]-x)
                    tmpy=abs(coord[1]-y)
                    tmpz=abs(coord[2]-z)
                    if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                        grid[rotation_idx+1,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += features[i]

        return grid
    
class PLIF:
    def __init__(self, PDB: str, MOL_SPLIT_START: int = 70, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super(PLIF,self).__init__()
        
        self.MOL_SPLIT_START=MOL_SPLIT_START
        self.pdb=PDB
        self.records=['ATOM']
        self.values=['HOH','CL','MG','ZN','MN','CA']
        self.ions=['CL','MG','ZN','MN','CA']
        self.interaction_slices={"hydrophobic":[0,1,6,7,8,9,10],
            "hbond":[0,1,7,11,13,15,16],
            "waterbridge":[0,1,[6,7],11,13,16,17],
            "saltbridge":[0,1,7,10,3,11,12],
            "pistacking":[0,1,7,11,6,12,13],
            "pication":[0,1,7,11,3,12,13],
            "halogen":[0,1,7,10,12,14,15],
            "metal":[0,1,11,8,6,17,16]} 

        self.column_names = ['RESNR', 'RESTYPE', 'DIST', 'LIG_IDX','PROT_IDX','FRAGMENT_ATOMS_COORDS', 'AA_COORDS']
        self.path = os.getcwd()
    

    # Divide a molecule into fragments
    def wholesome_pdb(self, mol, pdb):

        w = Chem.PDBWriter(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        w.write(frag)
        w.close()

        unwanted_entries= ['CONECT', 'END']            
        with open(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as oldfile, open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as newfile:
            for line in oldfile:
                if not any(unwanted_entry in line for unwanted_entry in unwanted_entries):
                    newfile.write(line)


        data = data2 = ""

        # Reading data from file1
        with open(f"ATOM_{pdb}.pdb") as fp:
            data = fp.read()

        # Reading data from file2

        with open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as fp:
            data2 = fp.read()

        # Merging 2 files
        # To add the data of file2
        # from next line
        #data += "\n"
        data += data2

        with open(f"HOH_{pdb}.pdb") as fp:
            data3 = fp.read()
        data += data3

        with open (f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as fp:
            fp.write(data)
        res_pdb_frags.append(f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        return res_pdb_frags #create_chain(res)


    def retreive_plip_interactions(self, pdb_file):
        """
        Retreives the interactions from PLIP.

        Parameters
        ----------
        pdb_file :
            The PDB file of the complex. 

        Returns
        -------
        dict :
            A dictionary of the binding sites and the interactions.
        """
        protlig = PDBComplex()   #instantiate the loader from PLIP
        protlig.load_pdb(pdb_file)   # load the pdb file
        for ligand in protlig.ligands:
            protlig.characterize_complex(ligand)   # find ligands and analyze interactions
        sites = {}
        # loop over binding sites
        for key, site in sorted(protlig.interaction_sets.items()):
            binding_site = BindingSiteReport(site)   # collect data about interactions
            # tuples of *_features and *_info will be converted to pandas DataFrame
            keys = (
                "hydrophobic",
                "hbond",
                "waterbridge",
                "saltbridge",
                "pistacking",
                "pication",
                "halogen",
                "metal"
            )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with 
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')
        # 2. information for each of these features, e.g. for hydrophobic
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')

            interactions = {
                k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
                for k in keys
            }
            sites[key] = interactions
        return sites

    def get_coords_prot(self, RESNR):
        ppdb = PandasPdb()
        ppdb.read_pdb(f"{self.pdb.split('.')[0]}_protein.pdb")
        only_protein=ppdb.df['ATOM']
        resnr_coords=[]
        for i in RESNR:
            resnr_coords.append(list(only_protein[only_protein['atom_number']==int(i)][['x_coord', 'y_coord', 'z_coord']].values[0]))
        return resnr_coords
    
    def interaction_df(self, split):

        all_interactions_df = pd.DataFrame()


        # We create the dictionary for the complex of interest:
        for idx, s in enumerate(split):

            pdb_id=s.split('.')[0]
            raw=pdb_id.split('_')[1]
            idx_frag=int(pdb_id.split('_')[2])
            interactions_by_site = self.retreive_plip_interactions(f"{pdb_id}.pdb")

            # Let’s see how many binding sites are detected:

    #         print(
    #             f"Number of binding sites detected in {pdb_id} : "
    #             f"{len(interactions_by_site)}\n"
    #             f"with {interactions_by_site.keys()}"
    #         )
            # In this case, the first binding site containing ligand 03P will be further investigated.
            index_of_selected_site = 0
            selected_site = list(interactions_by_site.keys())[index_of_selected_site]
            #print(selected_site)


            valid_types = [
                    "hydrophobic",
                    "hbond",
                    "waterbridge",
                    "saltbridge",
                    "pistacking",
                    "pication",
                    "halogen",
                    "metal",
                ]

            for _type in valid_types:
                output_df=self.create_df_from_binding_site(raw, interactions_by_site[selected_site], idx+self.MOL_SPLIT_START, selected_site,
                                                      interactions_by_site,
                                                      interaction_type=_type)
                all_interactions_df=all_interactions_df.append(output_df)
        all_interactions_df = all_interactions_df[all_interactions_df['RESNR'].notna()]
        all_interactions_df.to_csv(f"{self.path}/results_plifs/{raw}_plifs_and_properties.csv", index=False)
        return all_interactions_df


    # We can construct a pandas.DataFrame for a binding site and particular interaction type.

    def create_df_from_binding_site(self, raw, selected_site_interactions, fragment_idx, selected_site, 
                                    interactions_by_site, interaction_type="hbond"):
        """
        Creates a data frame from a binding site and interaction type.

        Parameters
        ----------
        selected_site_interactions : dict
            Precalculated interactions from PLIP for the selected site
        interaction_type : str
            The interaction type of interest (default set to hydrogen bonding).

        Returns
        -------
        pd.DataFrame :
            DataFrame with information retreived from PLIP.
        """
        # check if interaction type is valid:
        valid_types = [
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        ]


        if interaction_type not in valid_types:
            print("!!! Wrong interaction type specified. Hbond is chosen by default !!! \n")
            interaction_type = "hbond"

        def interaction_values(n):
            try:
                interactions=interactions_by_site[selected_site][interaction_type]
                if type(n) is list:
                    return [interactions[1:][x][i] for x in 
                        range(len(interactions[1:])) for i in n]
                else:
                    return [interactions[1:][x][n] for x in 
                        range(len(interactions[1:]))]
            except Exception:
                return None
            
        if interactions_by_site[selected_site][interaction_type][1:]:
            #print(list(map(interaction_values, self.interaction_slices[interaction_type])), self.column_names)
            selected_feats=list(map(interaction_values, self.interaction_slices[interaction_type]))
            #print(selected_feats)
            try: 
                if int(selected_feats[4])>int(selected_feats[3]):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3]  
            except: 
                if int(any(selected_feats[4]))>int(any(selected_feats[3])):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3] 
            df = pd.DataFrame(
                # data is stored AFTER the columns names
                [selected_feats],
                # column names are always the first element - we skipped that in the above - we are gonna use that for naming the df
                columns = self.column_names
            )

            df["INTERACTION_TYPE"]=interaction_type
            
            try:
                checked_coords=self.get_coords_prot(selected_feats[4][0].split(',') if ',' in selected_feats[4][0] \
                                                                   else selected_feats[4])
            except:
                checked_coords=selected_feats[6]
                
            df["AA_COORDS"]=[checked_coords]
                #[self.get_coords_prot(selected_feats[4].split(','))]
            df["FRAGMENT_ATOMS_COORDS"]=[selected_feats[5]]
                            #[self.get_coords_lig(selected_feats[3].split(','))]    
            df['FRAGMENT_ID']=fragment_idx

            # ideally we would like to exclude waters from further processing. Threrfore let us reduce any waterbridge 
            # interaction to the eucladean distance in order to omit water
            
            if interaction_type == "waterbridge":
                df['DIST']=[[np.linalg.norm(x) for x in df['DIST'].to_numpy()]]
                
            # also deal with one distance value and two coords, this is common in saltbridge interactions:
            if len(checked_coords) == len(selected_feats[2])*2:
                df['DIST']=[selected_feats[2] + selected_feats[2]]
                
        else:

            df= pd.DataFrame({'RESNR':[None], 'RESTYPE':[None], 'DIST':[None], 'LIG_IDX':[None],'PROT_IDX':[None],
                        'INTERACTION_TYPE':[interaction_type], "AA_COORDS": [None], "FRAGMENT_ATOMS_COORDS":[None],
                              'FRAGMENT_ID':[str(fragment_idx)]})



        return df
    
    def pdb_2_sdf(self, pdb):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("pdb", "sdf")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

        mol.AddHydrogens()


        obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
        return f"{pdb.split('.')[0]}.sdf"
    
    def sdf_2_pdb(self, sdf):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("sdf", "pdb")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

        mol.AddHydrogens()
        obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
        return f"HETATM_{sdf.split('.')[0]}.pdb"

    def save_bpdb(self, pdb,ppdb, record):  
        ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                    records=[record],
                    gz=False, 
                    append_newline=True)

    def get_HOH_pdb(self, pdb):
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb) 
        ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(self.values)]
        ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=['HETATM'],
                gz=False, 
                append_newline=True)

    def keep_relevant_hetatm(self, pdb):
        raw=str(self.pdb).split('.')[0]
        with open(pdb) as f1, open(f"ATOM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if 'ATOM' in line:
                    f2.write(line)
        with open(f'{raw}_ligand.pdb') as f1, open(f"HETATM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if ('HETATM' in line) and not any(ion in line for ion in self.ions):
                    f2.write(line)
        try: 
            self.get_HOH_pdb(pdb)
        except:
            with open(pdb) as f1, open(f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
                for line in f1:
                    if ('HETATM' in line) and any(ion in line for ion in self.ions):
                        f2.write(line)
        return
    
    
    def fragment_and_plif(self):
        path = os.getcwd()
        if not os.path.exists('results_plifs'):
            os.mkdir(f'{path}/results_plifs')

        raw=str(self.pdb).split('.')[0]
        self.sdf_2_pdb(f'{raw}_ligand.sdf')
        # output ATOM_{pdb_id}.pdb, # output HETATM_{pdb_id}.pdb # output HOH_{pdb_id}.pdb
        self.keep_relevant_hetatm(f'{raw}_protein.pdb')
        rdkit_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
        rdkit_mols_alt = Chem.MolFromMol2File(f'{raw}_ligand.mol2', sanitize=True, removeHs=True)
        content = open(f'{raw}_ligand.pdb').read()
        hets=re.findall("^HETATM (.*)", content, re.M)
        if len(hets)<5:
            # Read in the file
            with open(f'{raw}_ligand.pdb', 'r') as file :
                filedata = file.read()

            # Replace the target string
            filedata = filedata.replace('ATOM  ', 'HETATM')

            # Write the file out again
            with open(f'{raw}_ligand.pdb', 'w') as file:
                file.write(filedata)
        
        try: 
            rdkit_mols = Chem.RemoveHs(rdkit_mols[0])
            output_df = self.interaction_df(self.split_molecule(rdkit_mols,raw))
            
        except:  
            try:
                rdkit_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
                output_df = self.interaction_df(self.split_molecule(rdkit_mols,raw))
            except:
                try: 
                    output_df = self.interaction_df(self.split_molecule( Chem.MolFromMol2File(rdkit_mols_alt)))
                except:
                    try: 
                        rdkit_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
                        output_df = self.interaction_df(self.split_molecule(rdkit_mols,raw))
                    except:
                        try:
                            rdkit_mols = AllChem.MolFromPDBFile(f'HETATM_{raw}.pdb')
                            output_df = self.interaction_df(self.split_molecule(rdkit_mols,raw))
                        except:
    #                         raise Exception(f"Sorry, the pdb id: {raw} needs chekcing")
                            output_df= pd.DataFrame({'FRAGMENT_ID':[None], 'AA_COORDS':[None], 
                                                     'FRAGMENT_ATOMS_COORDS':[None], 
                                         'INTERACTION_TYPE':[None],'DIST':[None]})
                            print(f"Sorry, the pdb id: {raw} needs chekcing")
        os.chdir(f'{path}')

        return output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST'].agg(list)


def kd_equalizer (value):

    if 'mM' in value.split('=')[1]:
        return float(value.split('m')[0].split('=')[1]) / 1000
    elif 'uM' in value.split('=')[1]:
        return float(value.split('u')[0].split('=')[1]) / 1000000
    elif 'nM' in value.split('=')[1]:
        return float(value.split('n')[0].split('=')[1]) / 1000000000
    elif 'pM' in value.split('=')[1]:
        return float(value.split('p')[0].split('=')[1]) / 1000000000000
    elif 'fM' in value.split('=')[1]:
        return float(value.split('f')[0].split('=')[1]) / 1000000000000000

def dask_plif_cnn_train(row_pdb, row_kd):
    train_grids=None
    train_label=[]
    pdb_id = row_pdb
    print('pdb_id', pdb_id)

    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/{pdb_id}')

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
    
    if not len(df_plifSpecs) or df_plifSpecs['INTERACTION_TYPE'].apply(', '.join).unique()[0]=='metal':
        return

    train_label.extend([row_kd]*10)

    single_pdb_frags = []
    for idx, row in df_plifSpecs.iterrows():

        temp_plifs_prot={}
        temp_plifs_frag={}
        ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
        for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                           row['FRAGMENT_ATOMS_COORDS'],
                                                                                           row['INTERACTION_TYPE'],
                                                                                           row['DIST']):
            # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
            # for two amino acids or ligand atoms! Encoding by atom is crazy fun
            for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                             frag_lig_atm_coord_list):
                temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


        pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
        ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
        single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  

    grid=torch.zeros((full_batch,200,200,200,features_shape))

    for idx, mols in enumerate(single_pdb_frags):
        print(len(single_pdb_frags), idx)
#             threads = multiprocessing.Pool(len(single_pdb_frags))
#             threads.map(func, arg_list)

        coords1, features1 = Feature.get_features(mols[0],mols[2],1)
        coords2, features2 = Feature.get_features(mols[1],mols[3],0)

        # get the center point of protein
        center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
        coords=np.concatenate([coords1,coords2],axis = 0)
        features=np.concatenate([features1,features2],axis = 0)
        assert len(coords) == len(features)
        # zero the coordinates 
        coords = coords-center
        grid=Feature.grid(grid,coords,features,idx, rotation_bool=True)

    if train_grids is None:
        train_grids = grid.to_sparse()
    else:
        train_grids = torch.cat((train_grids,grid.to_sparse()), 0)
    print(train_grids.shape)

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    

    print("Memory utilised (bytes): ", sys.getsizeof(train_grids))
    with open('train_grids.pkl','wb') as f:
        pickle.dump(train_grids, f)
        
    #Save the label data of training and test set.
    with open('train_label.pkl','wb') as f:
        pickle.dump(train_label,f)    
        
    os.chdir(p_directory)
    
    return pdb_id


def dask_plif_cnn_test(row_pdb, row_kd):
    
    test_grids=None
    test_label=[]
    pdb_id = row_pdb
    print('pdb_id', pdb_id)

    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/{pdb_id}')

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
    
    if not len(df_plifSpecs):
        return
    
    test_label.extend([row_kd])

    single_pdb_frags = []
    print(len(single_pdb_frags))
    for idx, row in df_plifSpecs.iterrows():

        temp_plifs_prot={}
        temp_plifs_frag={}
        ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
        for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                           row['FRAGMENT_ATOMS_COORDS'],
                                                                                           row['INTERACTION_TYPE'],
                                                                                           row['DIST']):
            # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
            # for two amino acids or ligand atoms! Encoding by atom is crazy fun
            for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                             frag_lig_atm_coord_list):
                temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


        pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
        ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
        single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  

    grid=torch.zeros((1,200,200,200,features_shape))

    for idx, mols in enumerate(single_pdb_frags):
        coords1, features1 = Feature.get_features(mols[0],mols[2],1)
        coords2, features2 = Feature.get_features(mols[1],mols[3],0)

        # get the center point of protein
        center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
        coords=np.concatenate([coords1,coords2],axis = 0)
        features=np.concatenate([features1,features2],axis = 0)
        assert len(coords) == len(features)
        # zero the coordinates 
        coords = coords-center
        grid=Feature.grid(grid,coords,features,idx, rotation_bool=False)

    if test_grids is None:
        test_grids = grid.to_sparse()
    else:
        test_grids = torch.cat((test_grids,grid.to_sparse()), 0)
    print(test_grids.shape)

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    


    with open('test_grids.pkl','wb') as f:
        pickle.dump(test_grids, f)
        
    #Save the label data of training and test set.
    with open('test_label.pkl','wb') as f:
        pickle.dump(test_label,f)    
        
    os.chdir(p_directory)
    
    return pdb_id

if __name__ == "__main__":
    
    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/')

    Feature = Feature_extractor()

    p_directory = os.getcwd()

    dask_plif_cnn_train(str(sys.argv[1]), float(sys.argv[2]))

#     general=pd.read_csv('INDEX_general_PL_data.2020', sep=',')
#     refined=pd.read_csv('INDEX_refined_data.2020', sep=',')

#     general=general[general["Kd/Ki"].str.contains('IC|EC|>|<')==False]
#     refined=refined[refined["Kd/Ki"].str.contains('IC|EC|>|<')==False]

#     general["Kd/Ki"] = general["Kd/Ki"].str.replace('~','=')
#     refined["Kd/Ki"] = refined["Kd/Ki"].str.replace('~','=')


#     general['Kd/Ki']=general['Kd/Ki'].apply(lambda x: kd_equalizer(x))
#     refined['Kd/Ki']=refined['Kd/Ki'].apply(lambda x: kd_equalizer(x))

#     merged_PDBBind=general.append(refined) \
#                                 .sample(frac=1) \
#                                 .sample(frac=1) \
#                                 .reset_index(drop=True) \
#                                 .drop_duplicates(subset='PDB_code', keep="first") 

##    merged_PDBBind=pd.read_csv('merged_PDBBind.csv')
#     merged_PDBBind.rename(columns={'Kd/Ki': 'Kd_Ki'}, inplace=True)
#     merged_PDBBind[merged_PDBBind['PDB_code'].str.contains("3bho")==False]

##    train_df, test_df = train_test_split(merged_PDBBind, test_size=0.1)

#     merged_PDBBind.to_csv('merged_PDBBind.csv', index=False)
##    train_df.to_csv('train_df.csv', index=False)
##    test_df.to_csv('test_df.csv', index=False)


##    train_df.parallel_apply(lambda x: dask_plif_cnn_train(x.PDB_code, x.Kd_Ki), axis=1)
##    test_df.parallel_apply(lambda x: dask_plif_cnn_test(x.PDB_code, x.Kd_Ki), axis=1)



In [ ]:
pdbs= [glob.glob("*/*protein.pdb")]

In [2]:
from openbabel import pybel
ob = pybel.ob
import sys
import os
import glob

os.chdir('/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/')
values=['HOH','CL','MG','ZN','MN','CA', 'K']
list_aa=[]
list_atoms_ids=[]

print("hi")
for pdb in pdbs[0]:
    print(pdb)
    mol = next(pybel.readfile('pdb',pdb))
    obmol = mol.OBMol

    for res in ob.OBResidueIter(obmol):
        for atom in ob.OBResidueAtomIter(res):
            list_aa.append(res.GetName()) if res.GetName() not in list_aa and not res.IsHetAtom(atom) else list_aa
            list_atoms_ids.append(res.GetAtomID(atom)) if res.GetAtomID(atom) not in list_atoms_ids and not res.IsHetAtom(atom) else list_atoms_ids

hi
2lbv/2lbv_protein.pdb
1lt6/1lt6_protein.pdb
4eoh/4eoh_protein.pdb
3zc5/3zc5_protein.pdb
6hqy/6hqy_protein.pdb
1utl/1utl_protein.pdb
4lwi/4lwi_protein.pdb
3t4p/3t4p_protein.pdb
6oyz/6oyz_protein.pdb
5y13/5y13_protein.pdb
4i11/4i11_protein.pdb
5i2e/5i2e_protein.pdb
6n0q/6n0q_protein.pdb
2vvo/2vvo_protein.pdb
4tk1/4tk1_protein.pdb
1p1n/1p1n_protein.pdb
5l2w/5l2w_protein.pdb
2yay/2yay_protein.pdb
2y7i/2y7i_protein.pdb
1i7g/1i7g_protein.pdb
1nj5/1nj5_protein.pdb
5wei/5wei_protein.pdb
3l7d/3l7d_protein.pdb
4v04/4v04_protein.pdb
4qmz/4qmz_protein.pdb
6f3e/6f3e_protein.pdb
3vby/3vby_protein.pdb
4b7q/4b7q_protein.pdb
6qme/6qme_protein.pdb
1ft7/1ft7_protein.pdb
2xln/2xln_protein.pdb
2oz2/2oz2_protein.pdb
5ih9/5ih9_protein.pdb
5u0w/5u0w_protein.pdb
2ga2/2ga2_protein.pdb
6eox/6eox_protein.pdb
5ypp/5ypp_protein.pdb
5z95/5z95_protein.pdb
4a4l/4a4l_protein.pdb
4efk/4efk_protein.pdb
4us3/4us3_protein.pdb
4alv/4alv_protein.pdb
5he4/5he4_protein.pdb
5jrq/5jrq_protein.pdb
3k97/3k97_protein.pdb
3cvk/3c

4hiq/4hiq_protein.pdb
5k00/5k00_protein.pdb
4zsh/4zsh_protein.pdb
6bw2/6bw2_protein.pdb
3h0z/3h0z_protein.pdb
2wq5/2wq5_protein.pdb
3f17/3f17_protein.pdb
3ipa/3ipa_protein.pdb
4o72/4o72_protein.pdb
5d6p/5d6p_protein.pdb
1azx/1azx_protein.pdb
1ogx/1ogx_protein.pdb
2jkk/2jkk_protein.pdb
6jtc/6jtc_protein.pdb
2y68/2y68_protein.pdb
6mtu/6mtu_protein.pdb
1mh5/1mh5_protein.pdb
3k5d/3k5d_protein.pdb
3bm8/3bm8_protein.pdb
4ks3/4ks3_protein.pdb
4i9i/4i9i_protein.pdb
3iu9/3iu9_protein.pdb
4gd6/4gd6_protein.pdb
5c3k/5c3k_protein.pdb
6o9x/6o9x_protein.pdb
5al2/5al2_protein.pdb
2wxh/2wxh_protein.pdb
3cpb/3cpb_protein.pdb
3qrj/3qrj_protein.pdb
3t3d/3t3d_protein.pdb
2hzi/2hzi_protein.pdb
5i8g/5i8g_protein.pdb
4tq3/4tq3_protein.pdb
5l8y/5l8y_protein.pdb
6eg9/6eg9_protein.pdb
6rvl/6rvl_protein.pdb
4ygf/4ygf_protein.pdb
3s7m/3s7m_protein.pdb
4x5y/4x5y_protein.pdb
1a9q/1a9q_protein.pdb
4gki/4gki_protein.pdb
4umq/4umq_protein.pdb
3sow/3sow_protein.pdb
1o4k/1o4k_protein.pdb
6f9g/6f9g_protein.pdb
3d9n/3d9n_

3ral/3ral_protein.pdb
2baj/2baj_protein.pdb
1kzn/1kzn_protein.pdb
4a6v/4a6v_protein.pdb
3u7n/3u7n_protein.pdb
2wks/2wks_protein.pdb
1h46/1h46_protein.pdb
2pql/2pql_protein.pdb
8abp/8abp_protein.pdb
2h5j/2h5j_protein.pdb
5am7/5am7_protein.pdb
5ngb/5ngb_protein.pdb
6sen/6sen_protein.pdb
5oh1/5oh1_protein.pdb
6dgo/6dgo_protein.pdb
1qjb/1qjb_protein.pdb
2kfx/2kfx_protein.pdb
2rny/2rny_protein.pdb
6n0j/6n0j_protein.pdb
4j4v/4j4v_protein.pdb
5nz4/5nz4_protein.pdb
3ppj/3ppj_protein.pdb
4yve/4yve_protein.pdb
3ivc/3ivc_protein.pdb
6cdj/6cdj_protein.pdb
1o5p/1o5p_protein.pdb
2aq7/2aq7_protein.pdb
5mi5/5mi5_protein.pdb
5ovr/5ovr_protein.pdb
5oaj/5oaj_protein.pdb
3r5m/3r5m_protein.pdb
5tq6/5tq6_protein.pdb
6mzw/6mzw_protein.pdb
5hgc/5hgc_protein.pdb
2gh9/2gh9_protein.pdb
1dbb/1dbb_protein.pdb
4e5h/4e5h_protein.pdb
3pn4/3pn4_protein.pdb
1rdj/1rdj_protein.pdb
3c7n/3c7n_protein.pdb
1gbt/1gbt_protein.pdb
2w8f/2w8f_protein.pdb
6h7y/6h7y_protein.pdb
1nju/1nju_protein.pdb
4emr/4emr_protein.pdb
1ec1/1ec1_

4my6/4my6_protein.pdb
4zsa/4zsa_protein.pdb
3kf4/3kf4_protein.pdb
6dvn/6dvn_protein.pdb
2vwm/2vwm_protein.pdb
5w5k/5w5k_protein.pdb
1tys/1tys_protein.pdb
4u7q/4u7q_protein.pdb
3e7o/3e7o_protein.pdb
1yt7/1yt7_protein.pdb
4dma/4dma_protein.pdb
4u0c/4u0c_protein.pdb
4lh7/4lh7_protein.pdb
4x0u/4x0u_protein.pdb
5agu/5agu_protein.pdb
4lv1/4lv1_protein.pdb
4zx9/4zx9_protein.pdb
1lnm/1lnm_protein.pdb
5lwe/5lwe_protein.pdb
3qk0/3qk0_protein.pdb
4dr9/4dr9_protein.pdb
5vjp/5vjp_protein.pdb
4mro/4mro_protein.pdb
5j4y/5j4y_protein.pdb
6cc9/6cc9_protein.pdb
1rsi/1rsi_protein.pdb
4s1g/4s1g_protein.pdb
4egi/4egi_protein.pdb
4loj/4loj_protein.pdb
3hd3/3hd3_protein.pdb
6d5g/6d5g_protein.pdb
6hgf/6hgf_protein.pdb
4dkr/4dkr_protein.pdb
2vnp/2vnp_protein.pdb
1xxe/1xxe_protein.pdb
5tyn/5tyn_protein.pdb
6c9p/6c9p_protein.pdb
3gv6/3gv6_protein.pdb
1yy4/1yy4_protein.pdb
6cqz/6cqz_protein.pdb
2h96/2h96_protein.pdb
4utx/4utx_protein.pdb
4lm4/4lm4_protein.pdb
1cs4/1cs4_protein.pdb
6qlt/6qlt_protein.pdb
5alr/5alr_

6nk1/6nk1_protein.pdb
3v43/3v43_protein.pdb
1q0b/1q0b_protein.pdb
3q2a/3q2a_protein.pdb
5fat/5fat_protein.pdb
2qn1/2qn1_protein.pdb
3jwr/3jwr_protein.pdb
5db1/5db1_protein.pdb
5kj2/5kj2_protein.pdb
5mwo/5mwo_protein.pdb
5w2p/5w2p_protein.pdb
3tvc/3tvc_protein.pdb
2ei6/2ei6_protein.pdb
1pzo/1pzo_protein.pdb
5v6y/5v6y_protein.pdb
1ecq/1ecq_protein.pdb
4f6w/4f6w_protein.pdb
2ykb/2ykb_protein.pdb
4aa2/4aa2_protein.pdb
5zaj/5zaj_protein.pdb
4cga/4cga_protein.pdb
4m7j/4m7j_protein.pdb
3aig/3aig_protein.pdb
3p44/3p44_protein.pdb
4kb7/4kb7_protein.pdb
3hqh/3hqh_protein.pdb
5th2/5th2_protein.pdb
1owj/1owj_protein.pdb
1y2k/1y2k_protein.pdb
4rj8/4rj8_protein.pdb
2w6p/2w6p_protein.pdb
1vjc/1vjc_protein.pdb
4bkj/4bkj_protein.pdb
6poq/6poq_protein.pdb
4cl9/4cl9_protein.pdb
3cft/3cft_protein.pdb
6hky/6hky_protein.pdb
5sz5/5sz5_protein.pdb
3upk/3upk_protein.pdb
2qe4/2qe4_protein.pdb
6eya/6eya_protein.pdb
3qvv/3qvv_protein.pdb
6hys/6hys_protein.pdb
5m6h/5m6h_protein.pdb
5mnr/5mnr_protein.pdb
4kpx/4kpx_

...

5ldk/5ldk_protein.pdb
1jq8/1jq8_protein.pdb
4zs9/4zs9_protein.pdb
4clj/4clj_protein.pdb
1sqn/1sqn_protein.pdb
5orw/5orw_protein.pdb
2b1p/2b1p_protein.pdb
4qgd/4qgd_protein.pdb
4xoe/4xoe_protein.pdb
1kmv/1kmv_protein.pdb
4e1m/4e1m_protein.pdb
3owd/3owd_protein.pdb
4bie/4bie_protein.pdb
1u6q/1u6q_protein.pdb
6pml/6pml_protein.pdb
6uh0/6uh0_protein.pdb
5o87/5o87_protein.pdb
4cj4/4cj4_protein.pdb
3cyw/3cyw_protein.pdb
4jr5/4jr5_protein.pdb
6aom/6aom_protein.pdb
4qz6/4qz6_protein.pdb
3qtq/3qtq_protein.pdb
5m4c/5m4c_protein.pdb
4k6i/4k6i_protein.pdb
3bhx/3bhx_protein.pdb
6ic5/6ic5_protein.pdb
3ipy/3ipy_protein.pdb
4rvt/4rvt_protein.pdb
3l1s/3l1s_protein.pdb
5mzg/5mzg_protein.pdb
3pxz/3pxz_protein.pdb
3s9t/3s9t_protein.pdb
1j01/1j01_protein.pdb
4z22/4z22_protein.pdb
4j3d/4j3d_protein.pdb
9abp/9abp_protein.pdb
3wym/3wym_protein.pdb
2f7p/2f7p_protein.pdb
5agr/5agr_protein.pdb
4k43/4k43_protein.pdb
4cqg/4cqg_protein.pdb
6b1k/6b1k_protein.pdb
5zyj/5zyj_protein.pdb
1hyo/1hyo_protein.pdb
5zdb/5zdb_

2f1g/2f1g_protein.pdb
6b2p/6b2p_protein.pdb
3ovz/3ovz_protein.pdb
5v3h/5v3h_protein.pdb
4bam/4bam_protein.pdb
1bl4/1bl4_protein.pdb
5ngz/5ngz_protein.pdb
6ikm/6ikm_protein.pdb
4eoy/4eoy_protein.pdb
5nut/5nut_protein.pdb
3el1/3el1_protein.pdb
4s2i/4s2i_protein.pdb
4wyp/4wyp_protein.pdb
3gyn/3gyn_protein.pdb
6h7l/6h7l_protein.pdb
2phb/2phb_protein.pdb
4jx7/4jx7_protein.pdb
5cqj/5cqj_protein.pdb
6hov/6hov_protein.pdb
4wrb/4wrb_protein.pdb
2qi1/2qi1_protein.pdb
5jdc/5jdc_protein.pdb
1ok7/1ok7_protein.pdb
6hhh/6hhh_protein.pdb
6r8q/6r8q_protein.pdb
5mro/5mro_protein.pdb
4ktu/4ktu_protein.pdb
3faa/3faa_protein.pdb
2y5f/2y5f_protein.pdb
1i5d/1i5d_protein.pdb
5teh/5teh_protein.pdb
3w9k/3w9k_protein.pdb
3ud8/3ud8_protein.pdb
5ivf/5ivf_protein.pdb
4z84/4z84_protein.pdb
4c1e/4c1e_protein.pdb
4gxl/4gxl_protein.pdb
5lvx/5lvx_protein.pdb
5v1y/5v1y_protein.pdb
2ll7/2ll7_protein.pdb
3lmk/3lmk_protein.pdb
5zqr/5zqr_protein.pdb
4xsx/4xsx_protein.pdb
2zg1/2zg1_protein.pdb
5d3h/5d3h_protein.pdb
5fh8/5fh8_

3gc5/3gc5_protein.pdb
4q4r/4q4r_protein.pdb
3a4p/3a4p_protein.pdb
5cp9/5cp9_protein.pdb
5ehn/5ehn_protein.pdb
5wre/5wre_protein.pdb
5vcz/5vcz_protein.pdb
2f7i/2f7i_protein.pdb
6g36/6g36_protein.pdb
4xmo/4xmo_protein.pdb
1i43/1i43_protein.pdb
4y64/4y64_protein.pdb
5ktx/5ktx_protein.pdb
5y7h/5y7h_protein.pdb
4i7f/4i7f_protein.pdb
3g9l/3g9l_protein.pdb
1yfz/1yfz_protein.pdb
5ef8/5ef8_protein.pdb
5f1z/5f1z_protein.pdb
3er3/3er3_protein.pdb
2lto/2lto_protein.pdb
1jvu/1jvu_protein.pdb
4ayr/4ayr_protein.pdb
4xk9/4xk9_protein.pdb
3f6g/3f6g_protein.pdb
4hlk/4hlk_protein.pdb
3g76/3g76_protein.pdb
3t1a/3t1a_protein.pdb
2r3g/2r3g_protein.pdb
2g1q/2g1q_protein.pdb
4hze/4hze_protein.pdb
6gdy/6gdy_protein.pdb
2yex/2yex_protein.pdb
3fgc/3fgc_protein.pdb
6ee6/6ee6_protein.pdb
2wf5/2wf5_protein.pdb
1tuf/1tuf_protein.pdb
5ety/5ety_protein.pdb
3e3b/3e3b_protein.pdb
4ncm/4ncm_protein.pdb
6nzt/6nzt_protein.pdb
6nel/6nel_protein.pdb
6es0/6es0_protein.pdb
4q9o/4q9o_protein.pdb
5lgr/5lgr_protein.pdb
1uv5/1uv5_

5ak4/5ak4_protein.pdb
2wwj/2wwj_protein.pdb
2wbb/2wbb_protein.pdb
2cn0/2cn0_protein.pdb
2qp8/2qp8_protein.pdb
1nja/1nja_protein.pdb
5xmr/5xmr_protein.pdb
3mww/3mww_protein.pdb
5t4e/5t4e_protein.pdb
5myv/5myv_protein.pdb
3dp3/3dp3_protein.pdb
2vte/2vte_protein.pdb
5yg3/5yg3_protein.pdb
2c6m/2c6m_protein.pdb
1no9/1no9_protein.pdb
4nrl/4nrl_protein.pdb
2gvf/2gvf_protein.pdb
5wxg/5wxg_protein.pdb
1a1e/1a1e_protein.pdb
5lvq/5lvq_protein.pdb
6fby/6fby_protein.pdb
4jxw/4jxw_protein.pdb
1lke/1lke_protein.pdb
2qiq/2qiq_protein.pdb
6cpa/6cpa_protein.pdb
1okw/1okw_protein.pdb
4hy4/4hy4_protein.pdb
5arg/5arg_protein.pdb
5dry/5dry_protein.pdb
6g98/6g98_protein.pdb
3sue/3sue_protein.pdb
1oyq/1oyq_protein.pdb
1odi/1odi_protein.pdb
2fsa/2fsa_protein.pdb
3kqm/3kqm_protein.pdb
4c66/4c66_protein.pdb
4p0a/4p0a_protein.pdb
4iur/4iur_protein.pdb
2w6c/2w6c_protein.pdb
3gwx/3gwx_protein.pdb
3njq/3njq_protein.pdb
1rpa/1rpa_protein.pdb
2a0c/2a0c_protein.pdb
1ag9/1ag9_protein.pdb
4d2t/4d2t_protein.pdb
3zm6/3zm6_

4ufj/4ufj_protein.pdb
3hfz/3hfz_protein.pdb
6cqf/6cqf_protein.pdb
4qlu/4qlu_protein.pdb
3zpq/3zpq_protein.pdb
1b39/1b39_protein.pdb
2xrw/2xrw_protein.pdb
4i0s/4i0s_protein.pdb
3svj/3svj_protein.pdb
4xtv/4xtv_protein.pdb
3g2y/3g2y_protein.pdb
2qch/2qch_protein.pdb
6mdq/6mdq_protein.pdb
1ozv/1ozv_protein.pdb
3mo8/3mo8_protein.pdb
3djq/3djq_protein.pdb
4bnv/4bnv_protein.pdb
3krr/3krr_protein.pdb
1n1m/1n1m_protein.pdb
3dxk/3dxk_protein.pdb
3dcc/3dcc_protein.pdb
3n3l/3n3l_protein.pdb
1g7f/1g7f_protein.pdb
3gcu/3gcu_protein.pdb
4w9g/4w9g_protein.pdb
2wor/2wor_protein.pdb
3rse/3rse_protein.pdb
2cf8/2cf8_protein.pdb
5nka/5nka_protein.pdb
1ujj/1ujj_protein.pdb
5xst/5xst_protein.pdb
6g3v/6g3v_protein.pdb
5vt1/5vt1_protein.pdb
3dv5/3dv5_protein.pdb
1g50/1g50_protein.pdb
3tpr/3tpr_protein.pdb
5l6o/5l6o_protein.pdb
6kla/6kla_protein.pdb
5gv2/5gv2_protein.pdb
5q13/5q13_protein.pdb
4ec0/4ec0_protein.pdb
3bra/3bra_protein.pdb
5lny/5lny_protein.pdb
4uru/4uru_protein.pdb
5svz/5svz_protein.pdb
5ajo/5ajo_

In [37]:
from openbabel import pybel
ob = pybel.ob
import sys
import os
import glob

os.chdir('/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/')
values=['HOH','CL','MG','ZN','MN','CA', 'K']
list_aa=[]
list_atoms_ids=[]

print("hi")
for pdb in pdbs[0]:
    print(pdb)
    mol = next(pybel.readfile('pdb',pdb))
    obmol = mol.OBMol

    for res in ob.OBResidueIter(obmol):
        for atom, atom2 in zip(ob.OBResidueAtomIter(res),mol):
                print((atom.GetX(),atom.GetY(),atom.GetZ()), atom2.coords)

hi
2lbv/2lbv_protein.pdb
(6.859, 15.398, -18.225) (6.859, 15.398, -18.225)
(7.729, 15.072, -17.757) (7.729, 15.072, -17.757)
(6.077, 15.406, -17.539) (6.077, 15.406, -17.539)
(6.626, 14.758, -19.011) (6.626, 14.758, -19.011)
(7.067, 16.775, -18.754) (7.067, 16.775, -18.754)
(7.306, 17.452, -17.934) (7.306, 17.452, -17.934)
(5.792, 17.254, -19.44) (5.792, 17.254, -19.44)
(5.447, 18.433, -19.376) (5.447, 18.433, -19.376)
(8.226, 16.765, -19.753) (8.226, 16.765, -19.753)
(8.059, 15.984, -20.495) (8.059, 15.984, -20.495)
(8.284, 17.732, -20.252) (8.284, 17.732, -20.252)
(9.537, 16.494, -19.013) (9.537, 16.494, -19.013)
(9.521, 15.484, -18.603) (9.521, 15.484, -18.603)
(9.652, 17.213, -18.202) (9.652, 17.213, -18.202)
(10.923, 16.656, -20.165) (10.923, 16.656, -20.165)
(12.242, 16.263, -18.991) (12.242, 16.263, -18.991)
(12.229, 16.987, -18.176) (12.229, 16.987, -18.176)
(12.085, 15.261, -18.592) (12.085, 15.261, -18.592)
(13.205, 16.305, -19.5) (13.205, 16.305, -19.5)
(5.097, 16.331, -20.0

(-11.147, 1.692, 4.53) (6.859, 15.398, -18.225)
(-11.957, 2.004, 5.103) (7.729, 15.072, -17.757)
(-10.699, 0.308, 4.607) (6.077, 15.406, -17.539)
(-10.575, -0.069, 3.592) (6.626, 14.758, -19.011)
(-11.724, -0.556, 5.319) (7.067, 16.775, -18.754)
(-12.758, -0.072, 5.777) (7.306, 17.452, -17.934)
(-9.366, 0.239, 5.352) (5.792, 17.254, -19.44)
(-9.36, 0.974, 6.157) (5.447, 18.433, -19.376)
(-8.551, 0.452, 4.661) (8.226, 16.765, -19.753)
(-9.199, -1.062, 5.896) (8.059, 15.984, -20.495)
(-8.336, -1.108, 6.379) (8.284, 17.732, -20.252)
(-11.416, -1.842, 5.415) (6.859, 15.398, -18.225)
(-10.522, -2.181, 5.004) (7.729, 15.072, -17.757)
(-12.297, -2.794, 6.081) (6.077, 15.406, -17.539)
(-13.032, -2.238, 6.663) (6.626, 14.758, -19.011)
(-11.495, -3.69, 7.017) (7.067, 16.775, -18.754)
(-12.058, -4.341, 7.897) (7.306, 17.452, -17.934)
(-13.019, -3.651, 5.047) (5.792, 17.254, -19.44)
(-12.32, -4.372, 4.623) (5.447, 18.433, -19.376)
(-13.84, -4.183, 5.528) (8.226, 16.765, -19.753)
(-13.573, -2.769, 

(8.09, 3.979, -14.106) (6.859, 15.398, -18.225)
(7.828, 4.9, -14.512) (7.729, 15.072, -17.757)
(8.841, 3.936, -12.856) (6.077, 15.406, -17.539)
(8.35, 3.239, -12.176) (6.626, 14.758, -19.011)
(8.876, 5.312, -12.192) (7.067, 16.775, -18.754)
(9.209, 6.307, -12.826) (7.306, 17.452, -17.934)
(10.274, 3.472, -13.147) (5.792, 17.254, -19.44)
(10.243, 2.54, -13.711) (5.447, 18.433, -19.376)
(10.787, 4.234, -13.734) (8.226, 16.765, -19.753)
(11.033, 3.246, -11.838) (8.059, 15.984, -20.495)
(10.911, 4.117, -11.194) (8.284, 17.732, -20.252)
(10.633, 2.365, -11.335) (9.537, 16.494, -19.013)
(12.514, 3.034, -12.127) (9.521, 15.484, -18.603)
(12.876, 3.032, -13.292) (9.652, 17.213, -18.202)
(13.266, 2.874, -11.179) (10.923, 16.656, -20.165)
(8.524, 5.368, -10.911) (6.859, 15.398, -18.225)
(8.243, 4.495, -10.421) (7.729, 15.072, -17.757)
(8.524, 6.633, -10.176) (6.077, 15.406, -17.539)
(9.239, 7.289, -10.673) (6.626, 14.758, -19.011)
(8.972, 6.421, -8.736) (7.067, 16.775, -18.754)
(8.797, 5.351, -8

(-2.929, 0.886, -17.609) (6.859, 15.398, -18.225)
(-2.613, 0.135, -18.256) (7.729, 15.072, -17.757)
(-2.797, 2.275, -18.001) (6.077, 15.406, -17.539)
(-2.62, 2.337, -19.075) (6.626, 14.758, -19.011)
(-3.711, 2.814, -17.753) (7.067, 16.775, -18.754)
(-1.625, 2.889, -17.255) (7.306, 17.452, -17.934)
(-0.476, 2.763, -17.679) (5.792, 17.254, -19.44)
(-1.91, 3.511, -16.112) (6.859, 15.398, -18.225)
(-2.902, 3.585, -15.808) (7.729, 15.072, -17.757)
(-0.864, 4.091, -15.277) (6.077, 15.406, -17.539)
(0.104, 3.775, -15.666) (6.626, 14.758, -19.011)
(-1.01, 3.607, -13.839) (7.067, 16.775, -18.754)
(-1.742, 4.217, -13.059) (7.306, 17.452, -17.934)
(-0.946, 5.614, -15.311) (5.792, 17.254, -19.44)
(-1.974, 5.926, -15.128) (5.447, 18.433, -19.376)
(-0.299, 6.029, -14.539) (8.226, 16.765, -19.753)
(-0.496, 6.125, -16.681) (8.059, 15.984, -20.495)
(-1.035, 5.587, -17.461) (8.284, 17.732, -20.252)
(-0.714, 7.19, -16.758) (9.537, 16.494, -19.013)
(1.002, 5.903, -16.857) (9.521, 15.484, -18.603)
(1.663, 

(-3.871, -0.865, -2.798) (6.859, 15.398, -18.225)
(-3.005, -0.605, -3.311) (7.729, 15.072, -17.757)
(-4.116, -2.268, -2.469) (6.077, 15.406, -17.539)
(-4.781, -2.306, -1.606) (6.626, 14.758, -19.011)
(-2.814, -2.98, -2.11) (7.067, 16.775, -18.754)
(-1.834, -2.918, -2.851) (7.306, 17.452, -17.934)
(-4.775, -2.973, -3.659) (5.792, 17.254, -19.44)
(-5.746, -2.517, -3.854) (5.447, 18.433, -19.376)
(-4.947, -4.351, -3.359) (8.226, 16.765, -19.753)
(-5.524, -4.444, -2.56) (8.059, 15.984, -20.495)
(-3.888, -2.832, -4.895) (8.284, 17.732, -20.252)
(-2.917, -3.287, -4.698) (9.537, 16.494, -19.013)
(-4.361, -3.334, -5.739) (9.521, 15.484, -18.603)
(-3.755, -1.775, -5.127) (9.652, 17.213, -18.202)
(-2.822, -3.669, -0.97) (6.859, 15.398, -18.225)
(-3.687, -3.678, -0.392) (7.729, 15.072, -17.757)
(-1.653, -4.413, -0.506) (6.077, 15.406, -17.539)
(-0.821, -4.248, -1.191) (6.626, 14.758, -19.011)
(-2.0, -5.891, -0.453) (7.067, 16.775, -18.754)
(-3.007, -6.26, 0.152) (7.306, 17.452, -17.934)
(-1.25, -

(-16.386, 10.347, -0.218) (6.859, 15.398, -18.225)
(-16.48, 11.378, -0.117) (7.729, 15.072, -17.757)
(-17.402, 9.474, 0.362) (6.077, 15.406, -17.539)
(-16.903, 8.706, 0.953) (6.626, 14.758, -19.011)
(-18.214, 8.801, -0.74) (7.067, 16.775, -18.754)
(-18.617, 7.645, -0.608) (7.306, 17.452, -17.934)
(-18.335, 10.283, 1.273) (5.792, 17.254, -19.44)
(-18.535, 11.253, 0.817) (5.447, 18.433, -19.376)
(-19.273, 9.743, 1.401) (8.226, 16.765, -19.753)
(-17.676, 10.489, 2.639) (8.059, 15.984, -20.495)
(-17.594, 9.53, 3.151) (8.284, 17.732, -20.252)
(-16.681, 10.913, 2.502) (9.537, 16.494, -19.013)
(-18.526, 11.444, 3.478) (9.521, 15.484, -18.603)
(-19.54, 11.051, 3.555) (9.652, 17.213, -18.202)
(-18.553, 12.421, 2.995) (10.923, 16.656, -20.165)
(-17.96, 11.583, 4.816) (12.242, 16.263, -18.991)
(-16.979, 11.284, 4.989) (12.229, 16.987, -18.176)
(-18.675, 12.085, 5.816) (12.085, 15.261, -18.592)
(-19.907, 12.467, 5.611) (13.205, 16.305, -19.5)
(-20.328, 12.378, 4.664) (5.097, 16.331, -20.098)
(-20.

(1.535, 2.186, 3.783) (6.859, 15.398, -18.225)
1lt6/1lt6_protein.pdb
(31.915, 19.482, 0.125) (31.915, 19.482, 0.125)
(31.98, 18.924, -0.75) (31.98, 18.924, -0.75)
(32.109, 20.482, -0.085) (32.109, 20.482, -0.085)
(32.61, 19.13, 0.814) (32.61, 19.13, 0.814)
(30.54, 19.352, 0.696) (30.54, 19.352, 0.696)
(29.786, 19.702, -0.01) (29.786, 19.702, -0.01)
(30.518, 20.193, 1.973) (30.518, 20.193, 1.973)
(31.567, 20.397, 2.595) (31.567, 20.397, 2.595)
(30.206, 17.869, 0.995) (30.206, 17.869, 0.995)
(30.259, 17.293, 0.071) (30.259, 17.293, 0.071)
(30.924, 17.474, 1.713) (30.924, 17.474, 1.713)
(29.2, 17.801, 1.41) (29.2, 17.801, 1.41)
(29.339, 20.726, 2.357) (31.915, 19.482, 0.125)
(29.123, 21.565, 3.551) (31.98, 18.924, -0.75)
(29.69, 22.494, 3.502) (32.109, 20.482, -0.085)
(29.57, 20.883, 4.827) (32.61, 19.13, 0.814)
(29.686, 19.661, 4.865) (30.54, 19.352, 0.696)
(27.603, 21.742, 3.573) (29.786, 19.702, -0.01)
(27.127, 20.953, 4.156) (30.518, 20.193, 1.973)
(27.329, 22.713, 3.986) (31.567, 20.

(15.467, 29.181, 5.079) (31.915, 19.482, 0.125)
(15.014, 28.258, 5.237) (31.98, 18.924, -0.75)
(15.941, 29.938, 6.223) (32.109, 20.482, -0.085)
(16.887, 30.415, 5.967) (32.61, 19.13, 0.814)
(16.164, 28.986, 7.387) (30.54, 19.352, 0.696)
(15.623, 27.868, 7.409) (29.786, 19.702, -0.01)
(14.984, 31.068, 6.59) (30.518, 20.193, 1.973)
(15.488, 31.712, 7.311) (31.567, 20.397, 2.595)
(14.774, 31.635, 5.683) (30.206, 17.869, 0.995)
(13.671, 30.649, 7.186) (30.259, 17.293, 0.071)
(13.065, 30.178, 6.412) (30.924, 17.474, 1.713)
(13.858, 29.932, 7.985) (29.2, 17.801, 1.41)
(12.91, 31.832, 7.755) (29.339, 20.726, 2.357)
(11.818, 32.169, 7.289) (29.123, 21.565, 3.551)
(13.498, 32.483, 8.757) (29.69, 22.494, 3.502)
(13.036, 33.313, 9.18) (29.57, 20.883, 4.827)
(14.419, 32.161, 9.116) (29.686, 19.661, 4.865)
(17.004, 29.408, 8.322) (31.915, 19.482, 0.125)
(17.457, 30.338, 8.215) (31.98, 18.924, -0.75)
(17.305, 28.603, 9.49) (32.109, 20.482, -0.085)
(17.073, 27.576, 9.207) (32.61, 19.13, 0.814)
(16.47

(3.435, 15.369, 17.279) (31.915, 19.482, 0.125)
(2.896, 14.865, 18.012) (31.98, 18.924, -0.75)
(3.713, 14.667, 16.055) (32.109, 20.482, -0.085)
(4.17, 15.305, 15.299) (32.61, 19.13, 0.814)
(2.384, 14.161, 15.471) (30.54, 19.352, 0.696)
(1.547, 13.558, 16.152) (29.786, 19.702, -0.01)
(4.717, 13.525, 16.383) (30.518, 20.193, 1.973)
(5.692, 13.98, 16.559) (31.567, 20.397, 2.595)
(4.377, 13.034, 17.295) (30.206, 17.869, 0.995)
(4.888, 12.463, 15.325) (30.259, 17.293, 0.071)
(4.021, 11.377, 15.27) (30.924, 17.474, 1.713)
(3.226, 11.293, 16.01) (29.2, 17.801, 1.41)
(5.9, 12.548, 14.381) (29.339, 20.726, 2.357)
(6.599, 13.384, 14.41) (29.123, 21.565, 3.551)
(4.145, 10.396, 14.293) (29.69, 22.494, 3.502)
(3.448, 9.558, 14.268) (29.57, 20.883, 4.827)
(6.03, 11.565, 13.389) (29.686, 19.661, 4.865)
(6.819, 11.644, 12.641) (27.603, 21.742, 3.573)
(5.145, 10.485, 13.362) (27.127, 20.953, 4.156)
(5.27, 9.48, 12.429) (27.329, 22.713, 3.986)
(4.555, 8.809, 12.57) (27.224, 21.647, 2.137)
(2.17, 14.45, 

KeyboardInterrupt: 

In [4]:
list_aa

['MET',
 'THR',
 'VAL',
 'PRO',
 'ASP',
 'ARG',
 'SER',
 'GLU',
 'ILE',
 'ALA',
 'GLY',
 'LYS',
 'TRP',
 'TYR',
 'LEU',
 'ASN',
 'PHE',
 'CYS',
 'HIS',
 'GLN']

In [13]:
list_atoms_ids = [x.strip(' ') for x in list_atoms_ids]
list_atoms_ids=set(list_atoms_ids)

In [6]:
list_atoms_ids

[' N  ',
 ' HN1',
 ' HN2',
 ' HN3',
 ' CA ',
 ' HA ',
 ' C  ',
 ' O  ',
 ' CB ',
 ' HB1',
 ' HB2',
 ' CG ',
 ' HG1',
 ' HG2',
 ' SD ',
 ' CE ',
 ' HE1',
 ' HE2',
 ' HE3',
 ' H  ',
 ' HB ',
 ' OG1',
 ' CG2',
 '1HG2',
 '2HG2',
 '3HG2',
 ' CG1',
 '1HG1',
 '2HG1',
 '3HG1',
 ' CD ',
 ' HD1',
 ' HD2',
 ' OD1',
 ' OD2',
 ' NE ',
 ' HE ',
 ' CZ ',
 ' NH1',
 '1HH1',
 '2HH1',
 ' NH2',
 '1HH2',
 '2HH2',
 ' OG ',
 ' HG ',
 ' OE1',
 ' OE2',
 ' CD1',
 '1HD1',
 '2HD1',
 '3HD1',
 ' HB3',
 ' HA1',
 ' HA2',
 ' NZ ',
 ' HZ1',
 ' HZ2',
 ' HZ3',
 ' CD2',
 ' NE1',
 ' CE2',
 ' CE3',
 ' CZ2',
 ' CZ3',
 ' CH2',
 ' HH2',
 ' CE1',
 ' OH ',
 ' HH ',
 '1HD2',
 '2HD2',
 '3HD2',
 ' ND2',
 ' HZ ',
 ' SG ',
 ' ND1',
 ' NE2',
 '1HE2',
 '2HE2',
 ' OXT']

In [14]:
len(list_aa),len(list_atoms_ids)

(20, 81)